In [ ]:
!pip install ../input/efficientnet/EfficientNet-PyTorch -f ./ --no-index

In [ ]:
from PIL import Image
from tqdm import tqdm
import copy 
import pandas as pd
from torchvision import transforms, models
import torchvision
from torch import optim
from torch.optim import lr_scheduler
import torch
import os

from efficientnet_pytorch import EfficientNet

In [ ]:
transform_train = transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
        transforms.RandomResizedCrop(224),
        transforms.RandomAffine(degrees=10, translate=(0.2, 0.2), scale=(0.8, 1.2), shear=15),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(.2, .2, .2, .2),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
transform_valid = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])

In [ ]:
import random
with open('../input/plant-pathology-2021-fgvc8/train.csv', 'r') as f:
    csv = f.readlines()[1:]
    for _ in range(5):
        random.shuffle(csv)
    cnt = int(len(csv)*0.9)
    train_csv = csv[:cnt]
    valid_csv = csv[cnt:]

In [ ]:
label = {i.split(',')[1].strip() for i in train_csv}
label = sorted(label)
label2idx = {label:idx for idx, label in enumerate(label)}
label2idx

In [ ]:
class torchvision_Dataset(torch.utils.data.Dataset): 
    def __init__(self, data_root, csv, label, transforms=None):
        self.data = csv
        self.image_path = data_root
        self.label = label
        self.transform = transforms
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx): 
        image_name, label_name = self.data[idx].split(',')
        img = Image.open(os.path.join(self.image_path, image_name))
        if self.transform:
            x = self.transform(img)
        
        return x, self.label[label_name]

In [ ]:
train_dataset = torchvision_Dataset('train_images', train_csv, label2idx, transform_train)
valid_dataset = torchvision_Dataset('train_images', valid_csv, label2idx, transform_valid)

In [ ]:
train_dataloaders = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=1)
valid_dataloaders = torch.utils.data.DataLoader(valid_dataset, batch_size=16, shuffle=False, num_workers=1)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# model_ft = models.resnet34(pretrained=False)
model_ft = EfficientNet.from_name('efficientnet-b4', num_classes=12)
model_ft.to(device)

In [ ]:
class FocalLoss(torch.nn.Module):
    """
    The focal loss for fighting against class-imbalance
    """
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = 1e-12  # prevent training from Nan-loss error

    def forward(self, logits, target):
        """
        logits & target should be tensors with shape [batch_size, num_classes]
        """
        probs = torch.sigmoid(logits)
        one_subtract_probs = 1.0 - probs
        # add epsilon
        probs_new = probs + self.epsilon
        one_subtract_probs_new = one_subtract_probs + self.epsilon
        # calculate focal loss
        log_pt = target * torch.log(probs_new) + (1.0 - target) * torch.log(one_subtract_probs_new)
        pt = torch.exp(log_pt)
        focal_loss = -1.0 * (self.alpha * (1 - pt) ** self.gamma) * log_pt
        return torch.mean(focal_loss)

In [ ]:
criterion = FocalLoss()
optimizer_ft = optim.Adam(model_ft.parameters(), lr=1e-3)
exp_lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer_ft, 40, eta_min=1e-6, verbose=True)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    # since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        train_corrects = 0
        train_data_cnt = 0
        train_progress_bar = tqdm(train_dataloaders) 
        for inputs, labels in train_progress_bar:
            model.train()
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            train_corrects += torch.sum(preds == labels.data)
            train_data_cnt += inputs.size(0)
            train_progress_bar.set_description(f" Epoch[{epoch+1}/{num_epochs}] train : runing_Loss {running_loss / train_data_cnt:.5f}, train_acc {train_corrects / train_data_cnt:.5f}")
        scheduler.step()
 
        valid_corrects = 0
        valid_data_cnt = 0
        valid_progress_bar = tqdm(valid_dataloaders)
        for inputs, labels in valid_progress_bar:    
            model.eval()
           
            inputs = inputs.to(device)
            labels = labels.to(device)
           
            with torch.no_grad():
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                
            valid_corrects += torch.sum(preds == labels.data)
            valid_data_cnt += inputs.size(0)
            valid_progress_bar.set_description(f" Epoch[{epoch+1}/{num_epochs}] valid : valid_acc {valid_corrects / valid_data_cnt}")
            
        epoch_acc = valid_corrects / valid_dataset.__len__()
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_epoch = epoch
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model_ft.state_dict(), f"outputs/{best_epoch}.pth")
            print(f"best epoch : {best_epoch}")
    best_model_wts = copy.deepcopy(model.state_dict())        
    return best_model_wts

In [ ]:
#model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
#                       num_epochs=30)

In [ ]:
model_ft.load_state_dict(torch.load('../input/efficient-adam-focalloss/39.pth'))
model_ft.to(device)

In [ ]:
idx2label = {label2idx[label]:label for label in label2idx}

In [ ]:
from glob import glob
import csv
img_paths = glob("../input/plant-pathology-2021-fgvc8/test_images/*")

submit =[] 

for img_path in img_paths:
    model_ft.eval()
    img = Image.open(img_path)
    img = transform_valid(img)
    img = img.unsqueeze(0)
    with torch.no_grad():
        pred = model_ft(img.cuda())
    _, top_one = torch.max(pred, 1)
    img_path = img_path.split('/')[-1]
    
    submit.append([img_path,idx2label[int(top_one)]])
    
submission = pd.DataFrame(submit, columns=["image", "labels"])
submission.to_csv("/kaggle/working/submission.csv", index=False)